In [7]:
from transformers import BartForConditionalGeneration, BartTokenizer
import textwrap

In [8]:
def text_summarizer_from_text(text):
    introduction_index = text.find('# Introduction')
    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    if introduction_index != -1:
        introduction_text = text[introduction_index:]
        inputs = tokenizer.encode("summarize: " + introduction_text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=300, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        formatted_summary = "\n".join(textwrap.wrap(summary, width=80))

        return formatted_summary
    else:
        inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        formatted_summary = "\n".join(textwrap.wrap(summary, width=80))
        return formatted_summary

In [9]:
import csv
import sys

csv.field_size_limit(sys.maxsize)

input_csv_path = "150.csv"
output_csv_path = "output_data_150.csv"

with open(input_csv_path, 'r') as input_file, open(output_csv_path, 'w', newline='') as output_file:
    csv_reader = csv.reader(input_file)
    csv_writer = csv.writer(output_file)

    headers = next(csv_reader)
    headers.append("Summary")  
    csv_writer.writerow(headers)

    for i, row in enumerate(csv_reader, 1):
        try:
            text_data = row[2]  
            summary = text_summarizer_from_text(text_data) 
            row.append(summary)  
        except Exception as e:
            print(f"Error processing row {i}: {e}")
            summary = "no summary available"
            row.append(summary) 
        finally:
            csv_writer.writerow(row)  

print("Processing completed!")


Processing completed!
